In [1]:
from botorch.acquisition import LogExpectedImprovement
from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.models.transforms import Normalize, Standardize
from botorch.optim import optimize_acqf
from gpytorch.mlls import ExactMarginalLogLikelihood
from dataclasses import dataclass

import datetime
import polars as pl
import numpy as np
import torch

In [2]:
data = pl.read_csv('../data/tpch_q12_sweep.csv').rename({'': 'id'})
d = pl.read_json('../data/q12_result.json')
data = data.with_columns(mean_elapsed=pl.mean_horizontal('elapsed_0', 'elapsed_1', 'elapsed_2'))
data = data.with_columns(
    pl.col("receiptdate1").str.to_datetime("%Y-%m-%d", time_unit='ms'),
    pl.col("receiptdate2").str.to_datetime("%Y-%m-%d", time_unit='ms')
)
table_rd1 = pl.DataFrame(
    {
        'receiptdate1': d['sel_receiptdate1'][0].keys(),
        'selectivity1': d['sel_receiptdate1'][0].values()
    }
).with_columns(pl.col("receiptdate1").str.to_datetime("%Y-%m-%d", time_unit='ms'))
table_rd2 = pl.DataFrame(
    {
        'receiptdate2': d['sel_receiptdate2'][0].keys(),
        'selectivity2': d['sel_receiptdate2'][0].values()
    }
).with_columns(pl.col("receiptdate2").str.to_datetime("%Y-%m-%d", time_unit='ms'))
data = data.join(table_rd1, on='receiptdate1').join(table_rd2, on='receiptdate2')
data

id,receiptdate1,receiptdate2,elapsed_0,elapsed_1,elapsed_2,mean_elapsed,selectivity1,selectivity2
i64,datetime[ms],datetime[ms],f64,f64,f64,f64,f64,f64
0,1992-01-03 00:00:00,1992-01-03 00:00:00,0.006372,0.001012,0.000931,0.002771,1.0,0.0
1,1992-01-03 00:00:00,1992-02-02 00:00:00,0.231195,0.088566,0.088825,0.136195,1.0,0.000567
2,1992-01-03 00:00:00,1992-03-03 00:00:00,0.193612,0.157787,0.157573,0.169657,1.0,0.003764
3,1992-01-03 00:00:00,1992-04-02 00:00:00,0.201443,0.199245,0.201111,0.2006,1.0,0.010051
4,1992-01-03 00:00:00,1992-05-02 00:00:00,0.230072,0.228478,0.229138,0.229229,1.0,0.019428
…,…,…,…,…,…,…,…,…
7391,1998-12-27 00:00:00,1998-08-29 00:00:00,0.000568,0.000582,0.000557,0.000569,0.000004,0.978716
7392,1998-12-27 00:00:00,1998-09-28 00:00:00,0.000556,0.000566,0.000572,0.000564,0.000004,0.988609
7393,1998-12-27 00:00:00,1998-10-28 00:00:00,0.000571,0.000582,0.00056,0.000571,0.000004,0.995406


In [3]:
deviation_log = d['deviation_log'][0]
table_dev = pl.DataFrame({'deviation': d['deviation_log'][0]}).with_columns(pl.col("deviation").list.to_struct()).unnest("deviation")
table_dev = table_dev.rename(
        {'field_0': 'row1_id',
        'field_1': 'row2_id',
        'field_2': 'slope'}
    ).with_columns(pl.col('row1_id').cast(pl.Int64), pl.col('row2_id').cast(pl.Int64))
table_dev

row1_id,row2_id,slope
i64,i64,f64
0,1,49.143525
0,86,3.71849
0,87,23.907892
1,2,1.245692
1,86,182.73971
…,…,…
7390,7391,1.005479
7391,7392,1.007602
7392,7393,1.011122


In [4]:
class Tables:
    def __init__(self, table_data, table_dev):
        self.data = table_data
        self.dev = table_dev

    def closest_row_idx(self, date_a, date_b, table=None):
        dist = lambda col_a, col_b: abs(col_a - date_a) + abs(col_b - date_b)
        if table is None:
            table = self.data
        idx = table.select((dist(pl.col('receiptdate1'), pl.col('receiptdate2'))).arg_min()).item()
        return table[idx]['id'].item()
        
    def objfunc(self, date_a: datetime.datetime, date_b: datetime.datetime, rho=datetime.timedelta(days=30)):
        idx = self.closest_row_idx(date_a, date_b)
        dates_moved = (date_a + rho, date_b + rho)
        neighbors = self.data.join(self.dev.filter(pl.col('row1_id') == idx), left_on='id', right_on='row2_id')
        perturb_idx = self.closest_row_idx(dates_moved[0], dates_moved[1], neighbors)

        return self.dev.filter((pl.col('row1_id') == idx) & (pl.col('row2_id') == perturb_idx))[0]['slope'].item()

In [5]:
tables = Tables(data, table_dev)

In [6]:
# idx = tables.closest_row_idx(d1, d2)
# t = data.join(table_dev.filter(pl.col('row1_id') == idx), left_on='id', right_on='row2_id')
# perturb_idx = tables.closest_row_idx(d1, d2, t)
# table_dev.filter((pl.col('row1_id') == idx) & (pl.col('row2_id') == perturb_idx))[0]['slope']

# Training Initial GP

We'll select 10 starting points to train our GP model initially.

In [7]:
samples = (
    tables.data
    .sample(10)
    .with_columns(
        pl.struct('receiptdate1', 'receiptdate2')
        .map_elements(lambda row: tables.objfunc(row['receiptdate1'], row['receiptdate2']), return_dtype=pl.Float64)
        .alias('target')
    )
)
samples

id,receiptdate1,receiptdate2,elapsed_0,elapsed_1,elapsed_2,mean_elapsed,selectivity1,selectivity2,target
i64,datetime[ms],datetime[ms],f64,f64,f64,f64,f64,f64,f64
2523,1994-05-22 00:00:00,1994-05-22 00:00:00,0.097777,0.094962,0.095217,0.095985,0.669451,0.330549,1.000414
402,1992-05-02 00:00:00,1996-10-08 00:00:00,0.707456,0.705814,0.701288,0.704853,0.980572,0.69217,1.001481
1745,1993-08-25 00:00:00,1994-01-22 00:00:00,0.523571,0.402537,0.305702,0.410603,0.781618,0.280699,1.110342
4767,1996-07-10 00:00:00,1995-01-17 00:00:00,0.000595,0.000616,0.00059,0.0006,0.345229,0.430328,1.034799
4407,1996-03-12 00:00:00,1993-09-24 00:00:00,0.000648,0.000693,0.000719,0.000687,0.395114,0.23085,1.131596
2298,1994-02-21 00:00:00,1997-02-05 00:00:00,0.613757,0.649611,0.611688,0.625018,0.706843,0.742046,1.010462
7380,1998-12-27 00:00:00,1997-10-03 00:00:00,0.000573,0.000558,0.00057,0.000567,0.000004,0.841835,1.01177
4131,1995-12-13 00:00:00,1992-04-02 00:00:00,0.000652,0.000685,0.000627,0.000655,0.432507,0.010051,1.095624
3044,1994-11-18 00:00:00,1994-10-19 00:00:00,0.000599,0.000586,0.000592,0.000592,0.594611,0.39292,1.034703


In [27]:
train_x = samples.select('receiptdate1', 'receiptdate2').cast(pl.Float64).to_torch()
train_y = samples.select('target').to_torch()
train_x[0], train_y[0]

(tensor([7.6956e+11, 7.6956e+11], dtype=torch.float64),
 tensor([1.0004], dtype=torch.float64))

In [10]:
gp = SingleTaskGP(
    train_X=train_x,
    train_Y=train_y,
    input_transform=Normalize(d=len(train_x[0])),
    outcome_transform=Standardize(m=1),
)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)

ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (noise_prior): LogNormalPrior()
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SingleTaskGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (noise_prior): LogNormalPrior()
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): RBFKernel(
      (lengthscale_prior): LogNormalPrior()
      (raw_lengthscale_constraint): GreaterThan(2.500E-02)
    )
    (outcome_transform): Standardize()
    (input_transform): Normalize()
  )
)

### Acquisition Function

For now, as our GP estimates the slope from two queries we will directly try to maximize this value.

The bounds will allow us to control the area in which we search, this is where we can decide how to divide up the search space.

We can try something dumb where we search over the whole space for now

In [39]:
bounds = torch.stack([
    torch.concat([
        data.select(pl.col('receiptdate1', 'receiptdate2').min()).cast(pl.Float64).to_torch().flatten(), # minimum bounds for query 1
    ]),
    torch.concat([
        data.select(pl.col('receiptdate1', 'receiptdate2').max()).cast(pl.Float64).to_torch().flatten(), # maximum bounds for query 1
    ]),
]).to(torch.double)

logNEI = LogExpectedImprovement(model=gp, best_f=train_y.max())
candidate, acq_value = optimize_acqf(
    logNEI, bounds=bounds, q=1, num_restarts=1, raw_samples=100,
)
candidate = candidate.round().to(torch.int64) # Note we round off the item as we're dealing in ms
candidate, acq_value

(tensor([[751962887855, 753596844714]]), tensor(-4.4586, dtype=torch.float64))

Once we have the candidate we rebuild the query into something we could potentially pass into the DB

TODO: we will want to map the closest query to this for experimentation, however, we will need to round to the nearest whole number as we're dealing in milliseconds

In [40]:
# For some reason datetime does not take in ms, so we'll convert to seconds losing a little more precision
query = {
    'receiptdate1': datetime.datetime.fromtimestamp(candidate[0][0].item() // 1000), # datetime ms -> s
    'receiptdate2': datetime.datetime.fromtimestamp(candidate[0][1].item() // 1000), # datetime ms -> s
}
query

{'receiptdate1': datetime.datetime(1993, 10, 30, 2, 34, 47),
 'receiptdate2': datetime.datetime(1993, 11, 17, 23, 27, 24)}

## Gaining feedback by running on the DB

Now we have some queries to test, we can run it back on the DB to gain feedback on whether or not this worked out

In [41]:
import duckdb
import time

In [42]:
class DBWrapperTpchQ12:
    def __init__(self, db_path: str) -> None:
        con = duckdb.connect(database=db_path)
        con.execute('SET enable_progress_bar = false')
        self.con: duckdb.DuckDBPyConnection = con
        self.query_template = """
            SELECT
              l_shipmode,
              SUM(CASE
                  WHEN o_orderpriority = '1-URGENT'
                      OR o_orderpriority = '2-HIGH'
                      THEN 1
                  ELSE 0
              END) as high_line_count,
              SUM(CASE
                  WHEN o_orderpriority <> '1-URGENT'
                      AND o_orderpriority <> '2-HIGH'
                      THEN 1
                  ELSE 0
              END) AS low_line_count
            FROM
                orders,
                lineitem
            WHERE
              o_orderkey = l_orderkey
              AND l_shipmode IN ('AIR', 'REG AIR')
              AND l_commitdate < l_receiptdate
              AND l_shipdate < l_commitdate
              AND l_receiptdate >= $receiptdate1
              AND l_receiptdate < $receiptdate2
            GROUP BY
                  l_shipmode 
            ORDER BY
                l_shipmode;
        """

    def run_query(self, predicates: dict):
        start = time.time()
        self.con.sql(self.query_template, params=predicates)
        elapsed = time.time() - start
        
        return elapsed

In [43]:
def calculate_qerror(a, b):
    return max(a/b, b/a)

In [44]:
db = DBWrapperTpchQ12(db_path='../data/tpch_sf100.db')
perturb_query = {
    'receiptdate1': query['receiptdate1'] + datetime.timedelta(days=30),
    'receiptdate2': query['receiptdate2'] + datetime.timedelta(days=30),
}

In [45]:
a = db.run_query(query)
b = db.run_query(perturb_query)

In [46]:
calculate_qerror(a, b)

1.0371761330485236

# Doing the training loop

With one point of feedback, we can now do the whole training loop. Simply placing back the data point we just got into the original dataset and rebuilding the new GP model we can incorporate this new piece of data.